### Tools
Models can request to call tools that perform tasks such as fetching data from a database, searching the web, or running code. Tools are pairings of:
1. A schema, including the name of the tool, a description, and/or argument definitions (often a JSON schema)
2. A function or coroutine to execute.

In [1]:
import os
from langchain.chat_models import init_chat_model

os.environ["GROQ_API_KEY"] = os.getenv("GROQ_API_KEY")

model = init_chat_model("groq:qwen/qwen3-32b")
response = model.invoke("Why do parrots talk?")
response

AIMessage(content='<think>\nOkay, so I need to figure out why parrots talk. Let me start by recalling what I know about parrots. They\'re known for mimicking sounds, right? I\'ve heard that some species like African Greys and Budgerigars are especially good at it. But why do they do that? Is it just to imitate humans, or is there a deeper reason?\n\nFirst, maybe I should think about their natural behavior. In the wild, do they mimic other sounds? Probably. Birds often use calls to communicate with each other. So maybe in the wild, parrots use vocalizations to interact with their flock or to signal things like food, danger, or location. If they\'re in captivity, do they mimic humans because they\'re trying to fit in or communicate with their human companions? That makes sense. But why not just use their own calls?\n\nI remember reading that some birds have a structure called the syrinx, which allows them to produce a variety of sounds. Parrots might have a particularly developed syrinx,

In [2]:
from langchain.tools import tool

@tool
def get_weather(location:str)->str:
    """Get the weather at a location"""
    return f"It's sunny in {location}"


model_with_tools=model.bind_tools([get_weather])

In [3]:
response = model_with_tools.invoke("What's the weather like in Boston?")
print(response)
for tool_call in response.tool_calls:
    # View tool calls made by the model
    print(f"Tool: {tool_call['name']}")
    print(f"Args: {tool_call['args']}")

content='' additional_kwargs={'reasoning_content': 'Okay, the user is asking about the weather in Boston. I need to use the get_weather function. Let me check the function parameters. It requires a location, which in this case is Boston. I\'ll call the function with "Boston" as the location. Make sure the JSON is correctly formatted with the name and arguments. No other functions are available, so this should be straightforward.\n', 'tool_calls': [{'id': '0p6mw5e6b', 'function': {'arguments': '{"location":"Boston"}', 'name': 'get_weather'}, 'type': 'function'}]} response_metadata={'token_usage': {'completion_tokens': 102, 'prompt_tokens': 154, 'total_tokens': 256, 'completion_time': 0.173453929, 'completion_tokens_details': {'reasoning_tokens': 78}, 'prompt_time': 0.006937124, 'prompt_tokens_details': None, 'queue_time': 0.07070571, 'total_time': 0.180391053}, 'model_name': 'qwen/qwen3-32b', 'system_fingerprint': 'fp_d58dbe76cd', 'service_tier': 'on_demand', 'finish_reason': 'tool_call

### Tool Execution Loops

In [4]:
# Step 1: Model generates tool calls
messages = [{"role": "user", "content": "What's the weather in Boston?"}]
ai_msg = model_with_tools.invoke(messages)
messages.append(ai_msg)

# Step 2: Execute tools and collect results
for tool_call in ai_msg.tool_calls:
    # Execute the tool with the generated arguments
    tool_result = get_weather.invoke(tool_call)
    messages.append(tool_result)

# Step 3: Pass results back to model for final response
final_response = model_with_tools.invoke(messages)
print(final_response.text)
# "The current weather in Boston is 72°F and sunny."

The weather in Boston is sunny right now. It looks like a great day to enjoy outdoor activities!


In [5]:
messages

[{'role': 'user', 'content': "What's the weather in Boston?"},
 AIMessage(content='', additional_kwargs={'reasoning_content': 'Okay, the user is asking for the weather in Boston. I need to use the get_weather function. The function requires a location parameter. Boston is the location here. So I should call get_weather with location set to "Boston". Let me make sure there are no typos. Everything looks good. I\'ll format the tool call as specified.\n', 'tool_calls': [{'id': '1x8ftnmm0', 'function': {'arguments': '{"location":"Boston"}', 'name': 'get_weather'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 94, 'prompt_tokens': 153, 'total_tokens': 247, 'completion_time': 0.138260221, 'completion_tokens_details': {'reasoning_tokens': 70}, 'prompt_time': 0.007599063, 'prompt_tokens_details': None, 'queue_time': 0.070616126, 'total_time': 0.145859284}, 'model_name': 'qwen/qwen3-32b', 'system_fingerprint': 'fp_d58dbe76cd', 'service_tier': 'on_demand', 'finis